In [1]:
import torch
import sys
sys.path.append('../')
sys.path.append('../../')
from lstm_agent_cql_bc import DecisionLSTM
import yaml
import argparse
from tqdm import tqdm
import numpy as np
from tqdm import tqdm
import os

import matplotlib.pyplot as plt

import torch
import numpy as np

import pickle
from tqdm import tqdm
#import env_vizdoom2
import matplotlib.pyplot as plt
from itertools import count
import time
import random
from scipy.stats import sem

import torch
import numpy as np
import gym
from MinigridMemory.MinigridMemory_src.inference.val_minigridmemory import Minigrid, create_env

In [2]:
def load_model(seed, exp_name, loss_mode, stacked_input):
    agent = DecisionLSTM(4, 3, 128, mode='minigrid_memory')
    
    run_name = f'{exp_name}_{loss_mode}_{seed}_stacked_{stacked_input}'
    print(run_name)
    model_path = f'../ckpt/minigrid_memory_ckpt/{loss_mode}/{seed}/{run_name}.ckpt'
    
    agent.load_state_dict(torch.load(model_path))
    
    agent.eval()
    agent.to(agent.device)
    
    return agent

In [18]:
exp_name = 'minigrid_memory'
stacked_input = False
loss_mode = 'cql'

In [19]:
for length in np.arange(11, 101, 10):   
    totals = [] 

    for i in range(1, 3+1):
        agent = load_model(
            seed=i,
            exp_name=exp_name,
            stacked_input=stacked_input,
            loss_mode=loss_mode, 
        )

        _ = agent.eval()
        _ = agent.to(agent.device)
        PATH = f'../minigrid_memory_ckpt/{loss_mode}/{i}/{exp_name}_{loss_mode}_{i}_stacked_{stacked_input}.ckpt'

        device = 'cuda:0'
        EPISODE_TIMEOUT = 100 # 90
        env_name = {'type': 'Minigrid', 'name': 'MiniGrid-MemoryS13Random-v0'}

        NUMBER_OF_TRAIN_DATA = 100
        returns_red, returns_green, returns_total = [], [], []
        agent.eval()
        

        for j in tqdm(range(NUMBER_OF_TRAIN_DATA)):
            obsList, actList, rewList, doneList, isRedList = [], [], [], [], []
            env = create_env(env_name, length, render=True) # TODO: length 11 21 ... 91
            
            state = torch.tensor(env.reset(seed=j)).float() * 255.
            plt.close()
            state = state.reshape(1, 1, state.shape[0], state.shape[1], state.shape[2]).to(device)
            times = []
            # plt.imshow(obs['image'].transpose(1,2,0))
            # plt.show()
            # state = state.reshape(1, 1, state.shape[0], state.shape[1], state.shape[2])

            mask = torch.ones(1,1).to(device)
            done = False
            agent.init_hidden(1)
            action = 0
            rtg = 0.96

            for t in count():
                times.append(t)
                #result = policy(torch.from_numpy(obs['image']).unsqueeze(0).to(device), state, mask)
                #action, state = result['actions'], result['states']

                try:
                    states = torch.from_numpy(state).to(device).float()
                except:
                    states = state

                with torch.no_grad():
                    q_values = []
                    for possible_action in range(0, 4): 
                        action_tensor = torch.tensor([[[possible_action]]], 
                                                dtype=torch.float32, 
                                                device=device).long()
                        rtg_tensor = torch.tensor([[[rtg]]], 
                                                dtype=torch.float32, 
                                                device=device)#.long()
                        if loss_mode == 'cql':
                            update_lstm_hidden = possible_action==3
                        else:
                            update_lstm_hidden = True
                        
                        action_preds, q1, q2, _ = agent.forward(
                            states = states.float() / 255.,
                            actions = action_tensor,
                            returns_to_go = rtg_tensor,
                            update_hidden = update_lstm_hidden,
                            stacked_input = stacked_input,
                        )
                        q_value = torch.minimum(q1, q2)
                        q_values.append(q_value)

                        if not loss_mode == 'cql':
                            break

                    # Select action with max Q-value
                    if loss_mode == 'cql':
                        q_values = torch.cat(q_values, dim=-1)
                        action = torch.argmax(q_values).item() #+ 3
                    else:
                        action = torch.argmax(torch.softmax(action_preds, dim=-1).squeeze()).item()

                #action = random.choice([3,4])
                #print(t,action, q_values)
                state, reward, done, info = env.step((action, ))
                state = np.float32(state) * 255.
                state = state.reshape(1, 1, state.shape[0], state.shape[1], state.shape[2])
                rtg -= reward

                rewList.append(reward)
                actList.append(action)
                doneList.append(int(done))
                

                if done or t == EPISODE_TIMEOUT-1:
                    returns_total.append(np.sum(rewList))
                    break

        # print(f"\nResults for checkpoint {i}:")
        # print(f"Total average return:      {np.mean(returns_total):.2f}")
        # print("-" * 50)

        totals.append(np.mean(returns_total))


    print('\n')
    print(length)
    print('#'*50)

    print(f'TOTAL: {np.mean(totals)} ± {sem(totals)}')

minigrid_memory_cql_1_stacked_False


100%|██████████| 100/100 [00:59<00:00,  1.67it/s]


minigrid_memory_cql_2_stacked_False


100%|██████████| 100/100 [00:58<00:00,  1.71it/s]


minigrid_memory_cql_3_stacked_False


...

minigrid_memory_cql_2_stacked_False


100%|██████████| 100/100 [01:20<00:00,  1.24it/s]


minigrid_memory_cql_3_stacked_False


100%|██████████| 100/100 [01:39<00:00,  1.01it/s]



91
##################################################
TOTAL: 0.2405518737672584 ± 0.15757426332974417
